# Experiment: [TITLE]

**Date:** YYYY-MM-DD  
**Experiment ID:** `experiment_name`  
**Status:** In Progress / Complete / Failed  

---

## 1. Overview

### 1.1 Objective
[What are we testing? What hypothesis? What comparison?]

### 1.2 Key Results

| Metric | Value |
|--------|-------|
| **Primary Metric** | **X.XX** |
| Secondary Metric | Y.YY |

### 1.3 Conclusion
[1-2 sentence summary of findings]

---

## 2. Reproducibility Information

In [ ]:
import subprocess
import torch
from datetime import datetime

# Auto-capture reproducibility info
REPRODUCIBILITY_INFO = {
    'git_commit': subprocess.getoutput('git rev-parse HEAD'),
    'git_message': subprocess.getoutput('git log -1 --format="%s"'),
    'git_dirty': subprocess.getoutput('git status --porcelain') != '',
    'python_version': subprocess.getoutput('python --version').split()[1],
    'pytorch_version': torch.__version__,
    'cuda_version': torch.version.cuda if torch.cuda.is_available() else 'N/A',
    'gpu': torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'N/A',
    'random_seed': 42,  # UPDATE THIS
    'experiment_date': datetime.now().strftime('%Y-%m-%d'),
}

print('Reproducibility Information:')
for k, v in REPRODUCIBILITY_INFO.items():
    print(f'  {k}: {v}')

if REPRODUCIBILITY_INFO['git_dirty']:
    print('\n⚠️  WARNING: Uncommitted changes present!')

### Command to Reproduce

```bash
# Checkout correct commit
git checkout <COMMIT_HASH>

# Run experiment
python scripts/<SCRIPT>.py --args
```

---

## 3. Dataset

[Describe dataset, splits, any filtering]

In [ ]:
DATASET_INFO = {
    'total_cases': None,  # UPDATE
    'train_cases': None,
    'val_cases': None,
    'test_cases': None,
    'preprocessing_version': None,
}

for k, v in DATASET_INFO.items():
    print(f'{k}: {v}')

---

## 4. Model / Method

[Describe model architecture or method being tested]

In [ ]:
MODEL_CONFIG = {
    'architecture': None,  # UPDATE
    'parameters': None,
    # Add model-specific config
}

for k, v in MODEL_CONFIG.items():
    print(f'{k}: {v}')

---

## 5. Training Configuration

In [ ]:
TRAINING_CONFIG = {
    'max_epochs': None,  # UPDATE
    'batch_size': None,
    'learning_rate': None,
    'optimizer': None,
    'loss_function': None,
    # Add training-specific config
}

for k, v in TRAINING_CONFIG.items():
    print(f'{k}: {v}')

---

## 6. Results

### 6.1 Training Curves

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Publication-quality plot settings
plt.rcParams.update({
    'font.size': 12,
    'axes.labelsize': 14,
    'figure.figsize': (10, 6),
    'figure.dpi': 150,
    'savefig.dpi': 300,
})

# Load and plot metrics
# metrics = pd.read_csv('path/to/metrics.csv')
# plt.plot(...)
# plt.savefig('training_curves.png', dpi=300)

### 6.2 Quantitative Results

In [ ]:
RESULTS = {
    'best_metric': None,  # UPDATE
    'final_metric': None,
    'training_time': None,
}

print('Results:')
for k, v in RESULTS.items():
    print(f'  {k}: {v}')

### 6.3 Visualizations

[Add dose comparisons, DVH plots, gamma maps as needed]

---

## 7. Analysis

### 7.1 Observations
[What patterns do we see?]

### 7.2 Comparison to Previous Work
[How does this compare to baselines/prior experiments?]

### 7.3 Limitations
[What are the limitations of this experiment?]

---

## 8. Conclusions

[Summarize key takeaways]

---

## 9. Next Steps

- [ ] Next experiment 1
- [ ] Next experiment 2

---

## 10. Artifacts

| Artifact | Path |
|----------|------|
| Checkpoint | `runs/...` |
| Metrics | `runs/.../metrics.csv` |
| Config | `runs/.../training_config.json` |

---

*Notebook created: YYYY-MM-DD*  
*Last updated: YYYY-MM-DD*